In [1]:
import pandas as pd
import altair as alt
import eland as ed
import json
import numpy as np
import matplotlib.pyplot as plt
alt.data_transformers.disable_max_rows()
import os, sys
sys.path.append(os.path.join(os.getcwd(), '../src/'))
from data_reader.data_reader import DataReader
from elastic_manager.elastic_manager import ElasticManager
sys.path.append(os.path.join(os.getcwd(), '../src/analyze/'))
from analyze.h_one_extract_features import *
from analyze.analyze import *

Loading BokehJS ...

In [2]:
target = "20210701180000"
shots_data_index = "shots-" + target + "-data"
shots_meta_index = "shots-" + target + "-meta"

In [3]:
%%time
dr = DataReader()
shots_df = dr.multi_process_read_all(shots_data_index)

CPU times: user 22 s, sys: 3.45 s, total: 25.4 s
Wall time: 50.3 s


In [4]:
len(shots_df)

3150000

In [5]:
# ショットメタデータを確認
shots_meta_df = dr.read_shots_meta(shots_meta_index)
shots_meta_df.head()

,timestamp,shot_number,num_of_samples_in_cut_out,spm
0,2021-07-01T09:00:00,1,3000,20.0
1,2021-07-01T09:00:03,2,3000,20.0
2,2021-07-01T09:00:06,3,3000,20.0
3,2021-07-01T09:00:09,4,3000,20.0
4,2021-07-01T09:00:12,5,3000,20.0


In [6]:
print(f"ショット数：{len(shots_meta_df)}")

ショット数：1050


## 50サンプルごとに間引いたcontour図作成

In [7]:
import scipy
from scipy import signal

mask = np.array(
    [
        [1, 1, 1, 1, 1, 1, 1, 1, 1,],
        [1, 1, 1, 1, 1, 1, 1, 1, 1,],
        [1, 1, 1, 1, 1, 1, 1, 1, 1,],
        [1, 1, 1, 1, 1, 1, 1, 1, 1,],
        [1, 1, 1, 1, 1, 1, 1, 1, 1,],
        [1, 1, 1, 1, 1, 1, 1, 1, 1,],
        [1, 1, 1, 1, 1, 1, 1, 1, 1,],
        [1, 1, 1, 1, 1, 1, 1, 1, 1,],
        [1, 1, 1, 1, 1, 1, 1, 1, 1,],
    ]
)
mask = mask / 21

In [9]:
# exclude_shots=(983, 1227, 1228, 1229, 1369, 1381, 2894)
exclude_shots=()

## 荷重開始から破断までの間を対象とし、コンタ―図を作成する

In [10]:
from tqdm import tqdm

contour_index = "shots-" + target + "-contour"
ElasticManager.delete_index(index=contour_index)
ElasticManager.create_index(index=contour_index)

dr = DataReader()
start_index = "shots-" + target + "-start-point"
start_df = dr.read_all(start_index)
break_index = "shots-" + target + "-break-point"
break_df = dr.read_all(break_index)

for shot_number in tqdm(range(1, 11)): # 全ショットは重いので一部のショットのみ
    if shot_number in exclude_shots:
        continue
    
    shot_df = shots_df[shots_df.shot_number == shot_number].reset_index()
    # コンタ―図の対象範囲を荷重開始点および破断点から設定
    start_point = int(start_df[(start_df.shot_number == shot_number) & (start_df.load == "load01")].sequential_number_by_shot)
    # 破断ch (load01/02のセット or load03/04のセット)
    break_ch = "load01" if break_df[break_df.shot_number == shot_number].reset_index().break_channels[0][0] == "load01" else "load03"
    end_point = int(break_df[(break_df.shot_number == shot_number) & (break_df.load == break_ch)].sequential_number_by_shot)
    # コンター図にする間隔。暫定で80枚の図を作成。
    interval = (end_point - start_point) // 80
    # print(start_point, end_point, interval, break_ch)
    
    h_contor_array = []
    timestamp_by_fig = [] # 1枚の図毎のタイムスタンプ

    # 80枚のコンタ―図を生成
    for i in range(0, 80):
        timestamp_by_fig.append(shot_df.timestamp.loc[0])

        z = np.zeros((64, 64), dtype="float64")
        t = start_point + i * interval
        z[20, 20] = shot_df.load01[t]
        z[20, 40] = shot_df.load02[t]
        z[40, 20] = shot_df.load03[t]
        z[40, 40] = shot_df.load04[t]

        _a = scipy.signal.correlate2d(z, mask, mode="same", boundary="wrap")
        _a = scipy.signal.correlate2d(_a, mask, mode="same", boundary="wrap")
        _a = scipy.signal.correlate2d(_a, mask, mode="same", boundary="wrap")
        _a = scipy.signal.correlate2d(_a, mask, mode="same", boundary="wrap")
        _a = scipy.signal.correlate2d(_a, mask, mode="same", boundary="wrap")
        _a = scipy.signal.correlate2d(_a, mask, mode="same", boundary="wrap")
        _a = scipy.signal.correlate2d(_a, mask, mode="same", boundary="wrap")
        h_contor_array.append(_a)

    h_contor_array = np.array(h_contor_array)
    h_contor_array_reshaped = h_contor_array.reshape(80,-1)
    
    # create doc
    for i in range(0, 80):
        query = {
            "shot_number": shot_number,
            "number": i+1,
            "width": 64,
            "height": 64,
            "values": h_contor_array_reshaped[i]
        }
        doc_id = 80*(shot_number-1) + i
        ElasticManager.create_doc(index=contour_index, doc_id=doc_id, query=query)

100%|██████████| 10/10 [00:27<00:00,  2.71s/it]


## 特徴点ごとのコンタ―図

### 荷重開始点

In [12]:
from tqdm import tqdm

start_contour_index = "shots-" + target + "-start-contour"
ElasticManager.delete_index(index=start_contour_index)
ElasticManager.create_index(index=start_contour_index)

# 荷重開始点
dr = DataReader()
start_index = "shots-" + target + "-start-point"
start_df = dr.read_all(start_index)

for shot_number in tqdm(range(1, len(shots_meta_df)+1)):
    if shot_number in exclude_shots:
        continue
        
    shot_df = shots_df[shots_df.shot_number == shot_number].reset_index(drop=True)        
    # print(shot_df.head(3))
    start_in_shot_df = start_df[start_df.shot_number == shot_number].reset_index(drop=True)
#     from IPython.core.debugger import Pdb; Pdb().set_trace()    
    
    h_contor_array = []
    timestamp_by_ch = [] # ch毎のタイムスタンプ

    # ch毎に1枚、計4枚のコンタ―図を作成
    for target_ch in ["load01", "load02", "load03", "load04"]:
        df = start_in_shot_df[start_in_shot_df.load == target_ch].reset_index(drop=True)
        # 開始点のインデックス番号
        start_idx = int(df.sequential_number)
        # 開始点のインデックスにおける各chの荷重値
        channels_df = shot_df[shot_df.sequential_number == start_idx].reset_index(drop=True)
        load01 = float(channels_df.load01)
        load02 = float(channels_df.load02)
        load03 = float(channels_df.load03)
        load04 = float(channels_df.load04)    

        timestamp_by_ch.append(channels_df.timestamp.loc[0])
        
        z = np.zeros((64, 64), dtype="float64")
        z[20, 20] = load01
        z[20, 40] = load02
        z[40, 20] = load03
        z[40, 40] = load04

        _a = scipy.signal.correlate2d(z, mask, mode="same", boundary="wrap")
        _a = scipy.signal.correlate2d(_a, mask, mode="same", boundary="wrap")
        _a = scipy.signal.correlate2d(_a, mask, mode="same", boundary="wrap")
        _a = scipy.signal.correlate2d(_a, mask, mode="same", boundary="wrap")
        _a = scipy.signal.correlate2d(_a, mask, mode="same", boundary="wrap")
        _a = scipy.signal.correlate2d(_a, mask, mode="same", boundary="wrap")
        _a = scipy.signal.correlate2d(_a, mask, mode="same", boundary="wrap")
        h_contor_array.append(_a)

    h_contor_array = np.array(h_contor_array) # (1, 64, 64)
    h_contor_array_reshaped = h_contor_array.reshape(4,-1) # (4, 64*64=4096)
    # from IPython.core.debugger import Pdb; Pdb().set_trace()
    
    # create doc
    for i, load in enumerate(["load01", "load02", "load03", "load04"]):
        query = {
            "timestamp": timestamp_by_ch[i],
            "shot_number": shot_number,
            "load": load,
            "width": 64,
            "height": 64,
            "values": h_contor_array_reshaped[i]
        }
        doc_id = 4*(shot_number-1) + i
        ElasticManager.create_doc(index=start_contour_index, doc_id=doc_id, query=query)

100%|██████████| 1050/1050 [02:43<00:00,  6.43it/s]


### 最大荷重点

In [13]:
from tqdm import tqdm

max_contour_index = "shots-" + target + "-max-contour"
ElasticManager.delete_index(index=max_contour_index)
ElasticManager.create_index(index=max_contour_index)

dr = DataReader()
max_index = "shots-" + target + "-max-point"
max_df = dr.read_all(max_index)

for shot_number in tqdm(range(1, len(shots_meta_df)+1)): # 全ショットは重いので一部のショットのみ
    if shot_number in exclude_shots:
        continue
        
    shot_df = shots_df[shots_df.shot_number == shot_number].reset_index(drop=True)        
    # print(shot_df.head(3))
    max_in_shot_df = max_df[max_df.shot_number == shot_number].reset_index(drop=True)
#     from IPython.core.debugger import Pdb; Pdb().set_trace()    
    # ch毎に1枚、計4枚のコンタ―図を作成
    
    h_contor_array = []
    timestamp_by_ch = [] # ch毎のタイムスタンプ
    
    for target_ch in ["load01", "load02", "load03", "load04"]:
        df = max_in_shot_df[max_in_shot_df.load == target_ch].reset_index(drop=True)
        # 最大点のインデックス番号
        max_idx = int(df.sequential_number)
        # 最大点のインデックスにおける各chの荷重値
        channels_df = shot_df[shot_df.sequential_number == max_idx].reset_index(drop=True)
        load01 = float(channels_df.load01)
        load02 = float(channels_df.load02)
        load03 = float(channels_df.load03)
        load04 = float(channels_df.load04)    

        timestamp_by_ch.append(channels_df.timestamp.loc[0])
        
        z = np.zeros((64, 64), dtype="float64")
        z[20, 20] = load01
        z[20, 40] = load02
        z[40, 20] = load03
        z[40, 40] = load04

        _a = scipy.signal.correlate2d(z, mask, mode="same", boundary="wrap")
        _a = scipy.signal.correlate2d(_a, mask, mode="same", boundary="wrap")
        _a = scipy.signal.correlate2d(_a, mask, mode="same", boundary="wrap")
        _a = scipy.signal.correlate2d(_a, mask, mode="same", boundary="wrap")
        _a = scipy.signal.correlate2d(_a, mask, mode="same", boundary="wrap")
        _a = scipy.signal.correlate2d(_a, mask, mode="same", boundary="wrap")
        _a = scipy.signal.correlate2d(_a, mask, mode="same", boundary="wrap")
        h_contor_array.append(_a)

    h_contor_array = np.array(h_contor_array) # (1, 64, 64)
    h_contor_array_reshaped = h_contor_array.reshape(4,-1) # (4, 64*64=4096)

    for i, load in enumerate(["load01", "load02", "load03", "load04"]):
        query = {
            "timestamp": timestamp_by_ch[i],
            "shot_number": shot_number,
            "load": load,
            "width": 64,
            "height": 64,
            "values": h_contor_array_reshaped[i]
        }
        doc_id = 4*(shot_number-1) + i
        ElasticManager.create_doc(index=max_contour_index, doc_id=doc_id, query=query)

shots-20210701180000-max-contourが存在しません。
100%|██████████| 1050/1050 [02:38<00:00,  6.60it/s]


### 破断点

In [14]:
from tqdm import tqdm

break_contour_index = "shots-" + target + "-break-contour"
ElasticManager.delete_index(index=break_contour_index)
ElasticManager.create_index(index=break_contour_index)

dr = DataReader()
break_index = "shots-" + target + "-break-point"
break_df = dr.read_all(break_index)

for shot_number in tqdm(range(1, len(shots_meta_df)+1)): # 全ショットは重いので一部のショットのみ
    if shot_number in exclude_shots:
        continue
        
    shot_df = shots_df[shots_df.shot_number == shot_number].reset_index(drop=True)        
    break_in_shot_df = break_df[break_df.shot_number == shot_number].reset_index(drop=True)
#     from IPython.core.debugger import Pdb; Pdb().set_trace()    
    
    h_contor_array = []
    timestamp_by_ch = [] # ch毎のタイムスタンプ
    
    # ch毎に1枚、計4枚のコンタ―図を作成
    for target_ch in ["load01", "load02", "load03", "load04"]:
        df = break_in_shot_df[break_in_shot_df.load == target_ch].reset_index(drop=True)
        # 破断点のインデックス番号
        break_idx = int(df.sequential_number)
        # 破断点のインデックスにおける各chの荷重値
        channels_df = shot_df[shot_df.sequential_number == break_idx].reset_index(drop=True)
        load01 = float(channels_df.load01)
        load02 = float(channels_df.load02)
        load03 = float(channels_df.load03)
        load04 = float(channels_df.load04)    

        timestamp_by_ch.append(channels_df.timestamp.loc[0])
        
        z = np.zeros((64, 64), dtype="float64")
        z[20, 20] = load01
        z[20, 40] = load02
        z[40, 20] = load03
        z[40, 40] = load04

        _a = scipy.signal.correlate2d(z, mask, mode="same", boundary="wrap")
        _a = scipy.signal.correlate2d(_a, mask, mode="same", boundary="wrap")
        _a = scipy.signal.correlate2d(_a, mask, mode="same", boundary="wrap")
        _a = scipy.signal.correlate2d(_a, mask, mode="same", boundary="wrap")
        _a = scipy.signal.correlate2d(_a, mask, mode="same", boundary="wrap")
        _a = scipy.signal.correlate2d(_a, mask, mode="same", boundary="wrap")
        _a = scipy.signal.correlate2d(_a, mask, mode="same", boundary="wrap")
        h_contor_array.append(_a)

    h_contor_array = np.array(h_contor_array) # (1, 64, 64)
    h_contor_array_reshaped = h_contor_array.reshape(4,-1) # (4, 64*64=4096)
    # from IPython.core.debugger import Pdb; Pdb().set_trace() 

    # create doc
    for i, load in enumerate(["load01", "load02", "load03", "load04"]):
        query = {
            "timestamp": timestamp_by_ch[i],
            "shot_number": shot_number,
            "load": load,
            "width": 64,
            "height": 64,
            "values": h_contor_array_reshaped[i]
        }
        doc_id = 4*(shot_number-1) + i
        ElasticManager.create_doc(index=break_contour_index, doc_id=doc_id, query=query)

shots-20210701180000-break-contourが存在しません。
100%|██████████| 1050/1050 [02:39<00:00,  6.57it/s]


以下テンポラリ

In [ ]:
h_contor_array.shape
h_c_ = h_contor_array.reshape(80,-1)
h_c_[0].shape
#print(h_c_.min(), h_c_.max())

In [ ]:
for i in range(0, 80):
    query = {
        "number": i+1,
        "width": 64,
        "height": 64,
        "values": h_c_[i]
    }
    ElasticManager.create_doc(index="contour-001", doc_id=i, query=query)

In [ ]:
ElasticManager.delete_index(index="contour-001")